# Macroprudential Policy and the Housing Market - Replication Notebook
This notebook solves and simulates the Heterogenous Agent Housing Market (HAH) model and produces the output contained in my Master's thesis. 

**Dependencies:**

The code structure builds upon the framework developed by Jeppe Druedahl & Co. in the [NumEconCopenhagen Project](https://github.com/NumEconCopenhagen)

Packages required for running the notebooks are:
- [ConSav](https://pypi.org/project/ConSav/)
- [EconModel](https://pypi.org/project/EconModel/)
- [matplotlib](https://pypi.org/project/matplotlib/)
- [numpy](https://pypi.org/project/numpy/)
- [numba](https://pypi.org/project/numba/)
- [pandas](https://pypi.org/project/pandas/) ?
- [quantecon](https://pypi.org/project/quantecon/) ?

In [2]:
# imports
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

import numba as nb
nb.set_num_threads(4)

from HAHModel import HAHModelClass 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Baseline model

In [ ]:
model_baseline = HAHModelClass(
    name='baseline',
    par={
        'solmethod':'negm',
        #'cross_compute':True, # whether to cross-compute MPC if discrete choice is altered
        'do_print':True
        })

model_baseline.precompile_numba() # solve with very coarse grids
model_baseline.solve() 
model_baseline.simulate()